In [11]:
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.applications.xception import preprocess_input
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D
import h5py

数据特征提取

In [12]:
# 参考：https://keras.io/preprocessing/image/
# 参考：https://github.com/ypwhs/dogs_vs_cats
# 图片生成器
def get_img_xception_feature(batch_size_in=32,img_size_in =(299,299,3)):
    x_tensor = Input(img_size_in)
    
    """
    datagen =  ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=0.0,
        width_shift_range=0.0,
        height_shift_range=0.0,
        shear_range=0.0,
        zoom_range=0.0,
        horizontal_flip=False,
        vertical_flip = True,
        rescale=1. / 255,
        data_format='channels_last'  #形如（128,128,3）
    )
    """
    datagen =  ImageDataGenerator(preprocessing_function=preprocess_input)
    
    base_model = applications.Xception(input_tensor = x_tensor,weights='imagenet', include_top=False)
    
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output)) #添加GlobalAveragePooling2D层
    # 验证集暂时不与训练集分开
    # 图片预处理，参考：https://keras-cn.readthedocs.io/en/latest/preprocessing/image/
    train_generator = datagen.flow_from_directory("processed_train_data", img_size_in[:2], shuffle=False, 
                                              batch_size=batch_size_in, class_mode='categorical')
    
    test_generator = datagen.flow_from_directory("processed_test_data", img_size_in[:2], shuffle=False, 
                                             batch_size=batch_size_in, class_mode=None)
    # 参考 https://keras.io/models/model/
    train_features = model.predict_generator(train_generator, None,verbose=1) #train_generator.samples)
    test_features = model.predict_generator(test_generator, None,verbose=1) #test_generator.samples)
    
    test_file_que = []
    for i, fname in enumerate(test_generator.filenames):
        test_file_que.append(fname.encode('utf8'))
        

    with h5py.File("xception_bottleneck.h5") as h:
        h.create_dataset("train_features", data=train_features)
        h.create_dataset("test_features", data=test_features)
        h.create_dataset("train_label", data=train_generator.classes)
        h.create_dataset("test_file_que",data=test_file_que)

In [13]:
get_img_xception_feature()

Found 24998 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
391/391 [==============================] - 250s 639ms/step
